In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Concatenate, Reshape
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import preprocess_input
from PIL import Image 
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
df=pd.read_excel("Meme_Data_set.xlsx")

In [2]:

#Exploratory Data Analysis (EDA) Performance
# Display the first few rows of the  dataset
print("First few rows of the  dataset:")
print(df.head()) 

# Display the first few rows of the testing data
print("Last few rows of the  dataset:")
print(df.tail())  
# Display basic statistics of the  dataset
print("\nBasic statistics of the  dataset:")
print(df.describe()) 
# Display information about the  dataset
print("\nInformation about the  dataset:")
print(df.info()) 


First few rows of the  dataset:
                                            Captions  \
0  my mom when i play with my phone for 30 minute...   
1  Doing your own research for a test Copy and pa...   
2  Making original memes Following the idea of an...   
3  Upvote begging Upvoting your own meme on diffe...   
4                     Food delivery person Superhero   

                          Image URL                 Labels  
0  https://i.imgflip.com/30b1gx.jpg  Mom, Phone, Instagram  
1  https://i.imgflip.com/1ur9b0.jpg    Research, Wikipedia  
2  https://i.imgflip.com/24y43o.jpg       Original, Repost  
3  https://i.imgflip.com/22bdq6.jpg                 Upvote  
4    https://i.imgflip.com/9ehk.jpg    Delivery, Superhero  
Last few rows of the  dataset:
                                             Captions  \
44                                        Normal meme   
45  have a two tomoe sharingan having a eternal ma...   
46                          Schools Cancelled Student   
47    

In [3]:
#data preprocessing
class data_cleaning:
    #function to remove duplicates
    def remove_duplicates(self,data):        
        data.drop_duplicates(inplace=True)
    #function to handle missing values
    def handling_empty_cells(self,data):
        data.dropna(inplace=True)
obj=data_cleaning()
obj.remove_duplicates(df)
obj.handling_empty_cells(df)

In [4]:
"""def load_image(image_url):
    try:
        # Download image from URL
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img = img.convert('RGB')  # Ensure image is in RGB mode

        # Resize image to target size
        img = img.resize((224, 224))

        # Convert image to array and preprocess for VGG16
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)

        return img

    except Exception as e:
        print(f"Error loading image from {image_url}: {str(e)}")
        return None

# Preprocess all images in the DataFrame
processed_images = []

for image_url in df['Image URL']:
    processed_image = load_image(image_url)
    if processed_image is not None:
        processed_images.append(processed_image)

# Convert processed_images to a numpy array
X_images = np.vstack(processed_images)

print(f"Shape of preprocessed images: {X_images.shape}")
image_urls = df['Image URL'].tolist()
X_images = np.zeros((len(image_urls), 4096))  # assuming VGG16 output shape for fc2 layer"""


'def load_image(image_url):\n    try:\n        # Download image from URL\n        response = requests.get(image_url)\n        img = Image.open(BytesIO(response.content))\n        img = img.convert(\'RGB\')  # Ensure image is in RGB mode\n\n        # Resize image to target size\n        img = img.resize((224, 224))\n\n        # Convert image to array and preprocess for VGG16\n        img = img_to_array(img)\n        img = np.expand_dims(img, axis=0)\n        img = preprocess_input(img)\n\n        return img\n\n    except Exception as e:\n        print(f"Error loading image from {image_url}: {str(e)}")\n        return None\n\n# Preprocess all images in the DataFrame\nprocessed_images = []\n\nfor image_url in df[\'Image URL\']:\n    processed_image = load_image(image_url)\n    if processed_image is not None:\n        processed_images.append(processed_image)\n\n# Convert processed_images to a numpy array\nX_images = np.vstack(processed_images)\n\nprint(f"Shape of preprocessed images: {X_im

In [22]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import requests
from PIL import Image
from io import BytesIO

def load_image(image_url):
    try:
        # Download image from URL
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img = img.convert('RGB')  # Ensure image is in RGB mode

        # Resize image to target size expected by VGG16
        img = img.resize((224, 224))
        
        # Convert image to array and preprocess for VGG16
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)

        return img

    except Exception as e:
        print(f"Error loading image from {image_url}: {str(e)}")
        return None

def extract_vgg16_features(image_urls):
    processed_images = []

    for image_url in image_urls:
        processed_image = load_image(image_url)
        if processed_image is not None:
            processed_images.append(processed_image)

    X_images = np.vstack(processed_images)
    
    # Load VGG16 model without the top (fully connected layers)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Extract features using VGG16
    features = base_model.predict(X_images)

    # Flatten features to (number_of_images, 4096)
    features = features.reshape(len(processed_images), -1)

    return features

# Assuming df['Image URL'] contains your image URLs
X_images = extract_vgg16_features(df['Image URL'].tolist())

# Verify the shape of X_images after feature extraction
print(f"Shape of extracted features: {X_images.shape}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 50s 18s/step
Shape of extracted features: (49, 25088)


In [21]:

# Function to preprocess captions and labels
def preprocess_caption(caption):
    tokens = word_tokenize(caption.lower())
    return tokens

# Preprocess captions and build vocabulary
captions = df['Captions'].apply(preprocess_caption).tolist()
labels = df['Labels'].apply(lambda x: x.split(', ')).tolist()

#  Display preprocessed captions and labels
print("Preprocessed Captions:")
for caption in captions[:3]:  # Displaying the first 3 captions
    print(caption)

print("\nLabels:")
for label in labels[:3]:  # Displaying the first 3 sets of labels
    print(label)
# Create tokenizers for captions and labels
caption_tokenizer = tf.keras.preprocessing.text.Tokenizer()
caption_tokenizer.fit_on_texts(df["Captions"])
vocab_size = len(caption_tokenizer.word_index) + 1
print("Tokenizer Vocabulary:")
print(caption_tokenizer.word_index)

label_tokenizer = tf.keras.preprocessing.text.Tokenizer()
label_tokenizer.fit_on_texts(labels)
num_labels = len(label_tokenizer.word_index) + 1

# Generate sequences from tokens
caption_sequences = caption_tokenizer.texts_to_sequences(df["Captions"])
max_caption_length = max(len(seq) for seq in caption_sequences)

# Pad sequences to ensure uniform length
padded_caption_sequences = pad_sequences(caption_sequences, maxlen=max_caption_length, padding='post')

# Prepare data for model training

X_captions = np.array(padded_caption_sequences)
Y_captions = to_categorical(X_captions, num_classes=vocab_size)

# Convert labels to multi-hot encoded vectors
Y_labels = label_tokenizer.texts_to_matrix(labels, mode='binary')


print(f"Shape of X_images: {X_images.shape}")
print(f"Shape of X_captions: {X_captions.shape}")
print(f"Shape of Y_captions: {Y_captions.shape}")
print(f"Shape of Y_labels: {Y_labels.shape}")




Preprocessed Captions:
['my', 'mom', 'when', 'i', 'play', 'with', 'my', 'phone', 'for', '30', 'minutes', 'my', 'mom', 'when', 'she', 'looks', 'at', 'instagram', 'phone', 'for', 'two', 'hours']
['doing', 'your', 'own', 'research', 'for', 'a', 'test', 'copy', 'and', 'pasting', 'from', 'wikipedia']
['making', 'original', 'memes', 'following', 'the', 'idea', 'of', 'an', 'idiot', 'repost']

Labels:
['Mom', 'Phone', 'Instagram']
['Research', 'Wikipedia']
['Original', 'Repost']
Tokenizer Vocabulary:
{'the': 1, 'my': 2, 'a': 3, 'for': 4, 'when': 5, 'to': 6, 'i': 7, 'meme': 8, 'in': 9, 'with': 10, 'of': 11, 'school': 12, 'mom': 13, 'two': 14, 'and': 15, 'me': 16, 'play': 17, 'at': 18, 'from': 19, 'making': 20, 'upvoting': 21, 'on': 22, 'getting': 23, 'tell': 24, 'games': 25, 'how': 26, 'after': 27, 'corona': 28, 'have': 29, 'money': 30, 'toilet': 31, 'paper': 32, 'phone': 33, 'your': 34, 'own': 35, 'memes': 36, 'banned': 37, 'roblox': 38, 'saying': 39, 'surprise': 40, 'song': 41, 'minecraft': 4

In [18]:
# Define model
image_input = Input(shape=(25088,))
caption_input = Input(shape=(max_caption_length,))
label_input = Input(shape=(num_labels,))

image_features = Dense(256, activation='relu')(image_input)
caption_embedding = Embedding(vocab_size, 100, mask_zero=True)(caption_input)
caption_lstm = LSTM(256)(caption_embedding)

merged = Concatenate()([image_features, caption_lstm, label_input])
decoder = Dense(256, activation='relu')(merged)
output_caption = Dense(max_caption_length * vocab_size, activation='softmax')(decoder)  

output_caption = Reshape((max_caption_length, vocab_size))(output_caption)  

caption_model = Model(inputs=[image_input, caption_input, label_input], outputs=output_caption)

caption_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model Summary:")
print(caption_model.summary())



X_images_train, X_images_test, X_captions_train, X_captions_test, Y_labels_train, Y_labels_test, urls_train, urls_test, Y_captions_train, Y_captions_test = train_test_split(
    X_images, X_captions, Y_labels, df['Image URL'], Y_captions, test_size=0.2, random_state=42
)

# Train the model 
history = caption_model.fit(
    [X_images_train, X_captions_train, Y_labels_train], Y_captions_train,
    validation_data=([X_images_test, X_captions_test, Y_labels_test], Y_captions_test),
    epochs=10, batch_size=32
)

# Evaluate the model
loss, accuracy = caption_model.evaluate([X_images_test, X_captions_test, Y_labels_test], Y_captions_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')



Model Summary:


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)   │ (None, 29)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_13 (InputLayer)   │ (None, 25088)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_2 (Embedding)       │ (None, 29, 100)           │          25,100 │ input_layer_14[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal_2 (NotEqual)        │ (None, 29)                │               0 │ input_layer_14[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 256)               │       6,422,784 │ input_layer_13[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_2 (LSTM)                 │ (None, 256)               │         365,568 │ embedding_2[0][0],         │
│                               │                           │                 │ not_equal_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_15 (InputLayer)   │ (None, 65)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_2 (Concatenate)   │ (None, 577)               │               0 │ dense_6[0][0],             │
│                               │                           │                 │ lstm_2[0][0],              │
│                               │                           │                 │ input_layer_15[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 256)               │         147,968 │ concatenate_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 7279)              │       1,870,703 │ dense_7[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape_2 (Reshape)           │ (None, 29, 251)           │               0 │ dense_8[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 8,832,123 (33.69 MB)

 Trainable params: 8,832,123 (33.69 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 719ms/step - accuracy: 0.0221 - loss: 6.3652 - val_accuracy: 0.4655 - val_loss: 3.2860
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step - accuracy: 0.3905 - loss: 3.4439 - val_accuracy: 0.7793 - val_loss: 2.4489
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step - accuracy: 0.7310 - loss: 1.5770 - val_accuracy: 0.7759 - val_loss: 2.4098
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step - accuracy: 0.8115 - loss: 0.8913 - val_accuracy: 0.7586 - val_loss: 2.3475
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 0.8710 - loss: 0.5299 - val_accuracy: 0.7379 - val_loss: 2.3539
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step - accuracy: 0.9366 - loss: 0.3160 - val_accuracy: 0.7414 - val_loss: 2.4042
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step - accuracy: 0.9745 - loss: 0.1761 - val_accuracy: 0.7517 - val_loss: 2.4663
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step - accuracy: 0.9878 - loss: 0.1173 - val_accuracy: 0.7552 - val_

In [28]:
from IPython.display import display, Image as IPImage


predicted_captions = caption_model.predict([X_images_test, X_captions_test, Y_labels_test])

decoded_captions = []


for prediction in predicted_captions:
    
    predicted_words = []
    
    
    for timestep in prediction:
        
        predicted_word_index = np.argmax(timestep)
        
       
        predicted_word = caption_tokenizer.index_word.get(predicted_word_index, '')  
        
       
        predicted_words.append(predicted_word)
    
  
    decoded_caption = ' '.join(predicted_words)
    
   
    decoded_captions.append(decoded_caption)
print(predicted_words)
img_url=[]
for url in urls_test:
    img_url.append(url)
    
for i, caption in enumerate(decoded_captions):
    print(f"Predicted Caption: {caption}")
    display(IPImage(url=img_url[i]))
    print('-' * 50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
['money', 'pepperoni', '', '', '', '', '', '', '', '', '', '', '', 'family', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
Predicted Caption: money spotify                           


--------------------------------------------------
Predicted Caption: minecraft the freedom                          


--------------------------------------------------
Predicted Caption: money pepperoni paper                          


--------------------------------------------------
Predicted Caption: minecraft the freedom                          


--------------------------------------------------
Predicted Caption: money father freedom                          


...

--------------------------------------------------
Predicted Caption: food life freedom                          


--------------------------------------------------
Predicted Caption: money spotify freedom                          


--------------------------------------------------
Predicted Caption: food life freedom                          


--------------------------------------------------
Predicted Caption: money pepperoni            family               


--------------------------------------------------
